# 필기 한자 인식모델 구현 및 히라가나 변환

<div style="text-align: right"> 산업경영공학과 </div>
<div style="text-align: right"> 2018100922 </div>
<div style="text-align: right"> 이승건 </div>

참고) 경로는 사용자 환경에 맞게 설정하여 사용하도록 함.

## Colab에서 드라이브 연동하기

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

## Colab에서 GPU 이용하기

### 런타임 > 런타임 유형 변경 > GPU

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## 라이브러리

In [ ]:
!pip install bitstring

In [ ]:
import os
import time
import math
import re
import struct
import traceback
import codecs

import bitstring

import numpy as np
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers

from PIL import Image
import IPython.display as display

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import time

In [ ]:
#local library
import sys
sys.path.append('library')
import shared
from JIS0208 import JIS0208

## ETL 파일 읽기

In [ ]:
path = 'ETL/ETL8G/'

In [ ]:
file_list = [
    'ETL8G_01',
    'ETL8G_02',
    'ETL8G_03',
    'ETL8G_04',
    'ETL8G_05',
    'ETL8G_06',
    'ETL8G_07',
    'ETL8G_08',
    'ETL8G_09',
    'ETL8G_10',
    'ETL8G_11',
    'ETL8G_12',
    'ETL8G_13',
    'ETL8G_14',
    'ETL8G_15',
    'ETL8G_16',
    'ETL8G_17',
    'ETL8G_18',
    'ETL8G_19',
    'ETL8G_20',
    'ETL8G_21',
    'ETL8G_22',
    'ETL8G_23',
    'ETL8G_24',
    'ETL8G_25',
    'ETL8G_26',
    'ETL8G_27',
    'ETL8G_28',
    'ETL8G_29',
    'ETL8G_30',
    'ETL8G_31',
    'ETL8G_32',
    'ETL8G_33',
]

### 읽음 확인하기

In [ ]:
filename = file_list[0]
print(filename)
file_stream = bitstring.ConstBitStream(filename= path + filename)

In [ ]:
record = file_stream.readlist(','.join([
    'int:16', # Serial Sheet Number - [0]
    'uint:16', # JIS Character Code (JIS X 0208) - [1]
    '8*uint:8', # JIS Typical Reading (e.g. 'AI.MEDER') - [2:10]
    'int:32', # Serial Data Number - [10]
    '4*int:8', # - [11:15]
    '4*uint:16', # - [15:19]
    '2*int:8', # x and y - [19:21]
    'pad:240',
    'bytes:8128', # 16-graylevel (4-bit) image of 128 x 127 = 16256 pixels
    'pad:88',
]))

In [ ]:
print(type(record), len(record))
for idx, v in enumerate(record[:-1]):
    print(f'{idx} - {v}')

In [ ]:
record_dict = {
    'JIS_CODE': record[1],
    'image_data': record[-1]
}

In [ ]:
hex(record_dict['JIS_CODE'])

In [ ]:
width = 128
height = 127
img = np.array(Image.frombytes('F', (width, height), record_dict['image_data'], 'bit', 4))
plt.imshow(img)
plt.colorbar()

## X 데이터 추출

### 메모리 문제로 나눠서 추출 후 합침 

In [ ]:
with tf.device('/device:GPU:0'):
  total_samples = 0
  record_count = {}

  for filename in file_list:
      file_stream = bitstring.ConstBitStream(filename=path + filename)

      while True:
          try:
              record = file_stream.readlist(','.join([
                  'int:16', # Serial Sheet Number - [0]
                  'uint:16', # JIS Character Code (JIS X 0208) - [1]
                  '8*uint:8', # JIS Typical Reading (e.g. 'AI.MEDER') - [2:10]
                  'int:32', # Serial Data Number - [10]
                  '4*int:8', # - [11:15]
                  '4*uint:16', # - [15:19]
                  '2*int:8', # x and y - [19:21]
                  'pad:240',
                  'bytes:8128', # 16-graylevel (4-bit) image of 128 x 127 = 16256 pixels
                  'pad:88',
              ]))
          except:
              break
          
          total_samples += 1
          record_dict = {
              'JIS_CODE': record[1],
              'image_data': record[-1],
          }
          #합하는 부분 오류
          width = 128
          height = 127
          img = np.array(Image.frombytes('F', (width, height), record_dict['image_data'], 'bit', 4))
          scaled_img = (img * 17).astype(np.uint8)

          scaled_img_3 = scaled_img.reshape((1, *scaled_img.shape))


          if total_samples <= 10000:
            if total_samples == 1:
              x_train_1 = scaled_img_3.copy()
            else:
              print(x_train_1.shape)
              x_train_1 = np.append(x_train_1, scaled_img_3, axis = 0)

          elif total_samples <= 20000:
            if total_samples == 10001:
              x_train_2 = scaled_img_3.copy()
            else:
              print(x_train_2.shape)
              x_train_2 = np.append(x_train_2, scaled_img_3, axis = 0)

          elif total_samples <= 30000:
            if total_samples == 20001:
              x_train_3 = scaled_img_3.copy()
            else:
              print(x_train_3.shape)
              x_train_3 = np.append(x_train_3, scaled_img_3, axis = 0)
              
          elif total_samples <= 40000:
            if total_samples == 30001:
              x_train_4 = scaled_img_3.copy()
            else:
              print(x_train_4.shape)
              x_train_4 = np.append(x_train_4, scaled_img_3, axis = 0)

          elif total_samples <= 50000:
            if total_samples == 40001:
              x_train_5 = scaled_img_3.copy()
            else:
              print(x_train_5.shape)
              x_train_5 = np.append(x_train_5, scaled_img_3, axis = 0)

          elif total_samples <= 60000:
            if total_samples == 50001:
              x_train_6 = scaled_img_3.copy()
            else:
              print(x_train_6.shape)
              x_train_6 = np.append(x_train_6, scaled_img_3, axis = 0)
              
          elif total_samples <= 70000:
            if total_samples == 60001:
              x_train_7 = scaled_img_3.copy()
            else:
              print(x_train_7.shape)
              x_train_7 = np.append(x_train_7, scaled_img_3, axis = 0)

          elif total_samples <= 80000:
            if total_samples == 70001:
              x_train_8 = scaled_img_3.copy()
            else:
              print(x_train_8.shape)
              x_train_8 = np.append(x_train_8, scaled_img_3, axis = 0)

          elif total_samples <= 90000:
            if total_samples == 80001:
              x_train_9 = scaled_img_3.copy()
            else:
              print(x_train_9.shape)
              x_train_9 = np.append(x_train_9, scaled_img_3, axis = 0)
              
          elif total_samples <= 100000:
            if total_samples == 90001:
              x_train_10 = scaled_img_3.copy()
            else:
              print(x_train_10.shape)
              x_train_10 = np.append(x_train_10, scaled_img_3, axis = 0)

          elif total_samples <= 110000:
            if total_samples == 100001:
              x_train_11 = scaled_img_3.copy()
            else:
              print(x_train_11.shape)
              x_train_11 = np.append(x_train_11, scaled_img_3, axis = 0)

          elif total_samples <= 120000:
            if total_samples == 110001:
              x_train_12 = scaled_img_3.copy()
            else:
              print(x_train_12.shape)
              x_train_12 = np.append(x_train_12, scaled_img_3, axis = 0)
              
          elif total_samples <= 130000:
            if total_samples == 120001:
              x_train_13 = scaled_img_3.copy()
            else:
              print(x_train_13.shape)
              x_train_13 = np.append(x_train_13, scaled_img_3, axis = 0)

          elif total_samples <= 140000:
            if total_samples == 130001:
              x_train_14 = scaled_img_3.copy()
            else:
              print(x_train_14.shape)
              x_train_14 = np.append(x_train_14, scaled_img_3, axis = 0)

          elif total_samples <= 150000:
            if total_samples == 140001:
              x_train_15 = scaled_img_3.copy()
            else:
              print(x_train_15.shape)
              x_train_15 = np.append(x_train_15, scaled_img_3, axis = 0)

          elif total_samples <= 160000:
            if total_samples == 150001:
              x_train_16 = scaled_img_3.copy()
            else:
              print(x_train_16.shape)
              x_train_16 = np.append(x_train_16, scaled_img_3, axis = 0)
              

          
          name = hex(record_dict['JIS_CODE'])
          if name in record_count.keys():
              record_count[name] += 1
          else:
              record_count[name] = 1

In [ ]:
total_samples

In [ ]:
len(record_count.keys())

In [ ]:
record_count

In [ ]:
first_image = x_train_1[1]
first_image = np.array(first_image, dtype = 'float')
pixels = first_image.reshape((32,32))
plt.imshow(pixels, cmap = 'gray')
plt.show

### 데이터 합치기

In [ ]:
x_train_011 = np.append(x_train_1, x_train_2, axis = 0)
x_train_012 = np.append(x_train_3, x_train_4, axis = 0)
x_train_013 = np.append(x_train_5, x_train_6, axis = 0)
x_train_014 = np.append(x_train_7, x_train_8, axis = 0)
x_train_015 = np.append(x_train_9, x_train_10, axis = 0)
x_train_016 = np.append(x_train_11, x_train_12, axis = 0)
x_train_017 = np.append(x_train_13, x_train_14, axis = 0)
x_train_018 = np.append(x_train_15, x_train_16, axis = 0)

In [ ]:
print(x_train_011.shape)
print(x_train_012.shape)
print(x_train_013.shape)
print(x_train_014.shape)
print(x_train_015.shape)
print(x_train_016.shape)
print(x_train_017.shape)
print(x_train_018.shape)

In [ ]:
x_train_0011 = np.append(x_train_011, x_train_012, axis = 0)
x_train_0012 = np.append(x_train_013, x_train_014, axis = 0)
x_train_0013 = np.append(x_train_015, x_train_016, axis = 0)
x_train_0014 = np.append(x_train_017, x_train_018, axis = 0)

In [ ]:
print(x_train_0011.shape)
print(x_train_0012.shape)
print(x_train_0013.shape)
print(x_train_0014.shape)

In [ ]:
x_train_00011 = np.append(x_train_0011, x_train_0012, axis = 0)
x_train_00012 = np.append(x_train_0013, x_train_0014, axis = 0)

In [ ]:
print(x_train_00011.shape)
print(x_train_00012.shape)

In [ ]:
x_train = np.append(x_train_00011, x_train_00012, axis = 0)

In [ ]:
print(x_train.shape)

## Y 추출

### y_char (문자 list) 추출

In [ ]:
with tf.device('/device:GPU:0'):
  total_samples = 0
  record_count = {}
  
  y_train_char = np.array([])
  
  for filename in file_list:
      file_stream = bitstring.ConstBitStream(filename=path + filename)

      while True:
          try:
              record = file_stream.readlist(','.  join([
                  'int:16', # Serial Sheet Number - [0]
                  'uint:16', # JIS Character Code (JIS X 0208) - [1]
                  '8*uint:8', # JIS Typical Reading (e.g. 'AI.MEDER') - [2:10]
                  'int:32', # Serial Data Number - [10]
                  '4*int:8', # - [11:15]
                  '4*uint:16', # - [15:19]
                  '2*int:8', # x and y - [19:21]
                  'pad:240',
                  'bytes:8128', # 16-graylevel (4-bit) image of 128 x 127 = 16256 pixels
                  'pad:88',
              ]))
          except:
              break
          
          total_samples += 1
          record_dict = {
              'JIS_CODE': record[1],
              'image_data': record[-1],
          }
          
          if record_dict['JIS_CODE'] in JIS0208:
              unicode_char =   JIS0208[record_dict['JIS_CODE']]

          print(y_train_char.shape)
          y_train_char = np.append(y_train_char, [unicode_char])

### y (serial number) 추출

In [ ]:
with tf.device('/device:GPU:0'):
  total_samples = 0
  record_count = {}
  
  y_train = np.array([])
  
  for filename in file_list:
      file_stream = bitstring.ConstBitStream(filename=path + filename)

      while True:
          try:
              record = file_stream.readlist(','.  join([
                  'int:16', # Serial Sheet Number - [0]
                  'uint:16', # JIS Character Code (JIS X 0208) - [1]
                  '8*uint:8', # JIS Typical Reading (e.g. 'AI.MEDER') - [2:10]
                  'int:32', # Serial Data Number - [10]
                  '4*int:8', # - [11:15]
                  '4*uint:16', # - [15:19]
                  '2*int:8', # x and y - [19:21]
                  'pad:240',
                  'bytes:8128', # 16-graylevel (4-bit) image of 128 x 127 = 16256 pixels
                  'pad:88',
              ]))
          except:
              break
          
          total_samples += 1
          record_dict = {
              'JIS_CODE': record[1],
              'image_data': record[-1],
          }

          print(y_train.shape)
          y_train = np.append(y_train, [record[1]])

## train : test 나누기

### 111 : 50 로 test 추출

In [ ]:
len(np.unique(y_train))

In [ ]:
y_train_unique = np.unique(y_train)

In [ ]:
y_train_unique

In [ ]:
a = np.where(y_train == y_train_unique[0])

In [ ]:
x_test = np.array([])
y_test = np.array([])
f = 0
for i in range(len(y_train_unique)):
  find_list = np.where(y_train == y_train_unique[i])
  t = 0
  for j in range(len(find_list[0])):
    if t < 50:
      #y_test 분배 50개
      y_test = np.append(y_test, [y_train_unique[i]])


      #x_test 분배 50개
      find_x_list = x_train[find_list[0][j]]
      find_x_list_3 = find_x_list.reshape((1, *find_x_list.shape))
      if t == 0:
        x_test_mid = find_x_list_3.copy()
      else:
        x_test_mid = np.append(x_test_mid, find_x_list_3, axis = 0)



    t+=1

  if f == 0:
    x_test = x_test_mid.copy()
    f+=1
  else:
    print(x_test.shape)
    x_test = np.append(x_test, x_test_mid, axis = 0)

In [ ]:
x_test.shape

In [ ]:
y_test.shape

### train에서 test 값 삭제

In [ ]:
x_train = np.delete(x_train, np.s_[0:47800], axis = 0)
y_train = np.delete(y_train, np.s_[0:47800], axis = 0)
y_train_char = np.delete(y_char, np.s_[0:47800], axis = 0)

In [ ]:
print(y_train.shape)
print(y_train.shape)

## 히라가나 지우기

In [ ]:
hiragana = ['あ', 'い', 'う', 'え', 'お', 'か', 'き', 'く', 'け', 'こ', 'さ', 'し', 'す', 'せ', 'そ', 'た', 'ち', 'つ', 'て', 'と', 'な',
            'に', 'ぬ', 'ね', 'の', 'は', 'ひ', 'ふ', 'へ', 'ほ', 'ま', 'み', 'む', 'め', 'も', 'や', 'ゆ', 'よ', 'ら', 'り', 'る', 'れ',
            'ろ', 'わ', 'を', 'ん', 'ぎ', 'が', 'ぐ', 'げ', 'ご', 'ざ', 'じ', 'ず', 'ぜ', 'ぞ', 'だ', 'ぢ', 'づ', 'で', 'ど', 'ば', 'び',
            'ぶ', 'べ', 'ぼ', 'ぱ', 'ぴ', 'ぷ', 'ぺ', 'ぽ', 'ゃ', 'ゅ', 'ょ', 'っ']

In [ ]:
len(hiragana)

In [ ]:
add_hiragana_train = []
for i in range(len(hiragana)):
  add_hiragana_train = np.append(add_hiragana_train, np.where(y_train_char == hiragana[i]))

In [ ]:
add_hiragana_train = add_hiragana_train.astype('int')

In [ ]:
x_train = np.delete(x_train, add_hiragana_train, axis = 0)
y_train = np.delete(y_train, add_hiragana_train, axis = 0)
x_test = np.delete(x_test, np.s_[:3750], axis = 0)
y_test = np.delete(y_test, np.s_[:3750], axis = 0)

In [ ]:
y_train_char = np.delete(y_train_char, add_hiragana_train, axis = 0)

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

## 바뀐 Y 인덱스 재정렬

In [ ]:
len(np.unique(y_test))

In [ ]:
k = 0
y_unique = np.unique(y_test).copy()
for j in range(len(y_unique)):
    for i in range(len(y_test)):
        if y_test[i] == y_unique[j]:
            y_test[i] = k
    for i in range(len(y_train)):
        if y_train[i] == y_unique[j]:
            y_train[i] = k
    k += 1

## 모델 저장

In [ ]:
np.save('data/y_train.npy', x_train)
np.save('data/y_test.npy', x_test)
np.save('data/y_train.npy', y_train)
np.save('data/y_test.npy', y_test)
np.save('data/y_train_char.npy', y_train_char)

## 모델 학습

In [ ]:
x_train = np.load('data/x_train.npy')
y_train = np.load('data/y_train.npy')
x_test = np.load('data/x_test.npy')
y_test = np.load('data/y_test.npy')

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
len(np.unique(y_test))

In [ ]:
num_classes = 881; epochs = 20; batch_size = 128
learning_rate = 0.001;

In [ ]:
img_rows, img_cols = 127, 128

In [ ]:
input_shape = (img_rows, img_cols, 1)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

### 메모리 문제로 정규화 진행하지 않음

In [ ]:
#정규화 하면 너무 크기가 커지는듯
# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255

# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
with tf.device('/device:GPU:0'):
  model = tf.keras.Sequential()

  model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape = input_shape))
  model.add(layers.Conv2D(32, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(Dropout(0.25))

  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(Dropout(0.25))

  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(Dropout(0.25))
  
  model.add(layers.Flatten())

  model.add(layers.Dense(2048, activation = 'relu'))
  model.add(Dropout(0.25))

  model.add(layers.Dense(1024, activation = 'relu'))

  model.add(layers.Dense(num_classes, activation = 'softmax'))

  model.compile(loss = tf.keras.losses.categorical_crossentropy,
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
                metrics = ['accuracy'])
  early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 5)
  hist = model.fit(x_train, y_train,
                  batch_size = batch_size, epochs = epochs,
                  validation_split = 0.2, callbacks = [early_stop]
                  )

  score = model.evaluate(x_test, y_test, verbose=1)

  print(' - test_loss:', score[0], ' - test_acc:', score[1])
  #model.save("cnn_japanese.h5")

### 모델 저장

In [ ]:
model.save("model/cnn_japanese_96_19.h5")

### 모델 확인

In [ ]:
model.summary()

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots(figsize = (15,5))
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label = 'train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label = 'val loss')
acc_ax.plot(hist.history['accuracy'], 'b', label = 'train acc')
acc_ax.plot(hist.history['val_accuracy'], 'g', label = 'val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuracy')
loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()